In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [8]:
path = get_file(
    'russian_text.txt',
      origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
import codecs
with codecs.open('russian_text.txt', "r", "utf_8_sig") as f:
    text = f.read().lower()
print('corpus length:', len(text))
remove_chars = ['\n', '!', '(', ')', '*', ',', '-', '.', ':', ';', '?', '–', '«', '»', '“', '”', '…', '"', '=', '[', ']', '\u200b', '—', '’', '„',
                 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '°', 'à', 'ä', 'è', 'é', 'ê', 'ó', 'ö', 'ü', 'ł', '́',
                 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
for i in remove_chars :
    text = text.replace(i, '')
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 1216493
total chars: 34


In [9]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 379447
Vectorization...


In [10]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [11]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [12]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [13]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [14]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
379447/379447 [==============================] - 434s 1ms/step - loss: 1.9495

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "редствами доискиваться свидания он уверя"
редствами доискиваться свидания он уверян сколько в нем вот вот все в собой совсем не совсем в собой случае совсем в собой серденно совсем в собой в середней как в собой случае на сей от вот от стало приступить в себя приговорил он вот все совсем в середный случился последнее вот вот все с совсем вы все это все с сейство только послушал он приговорил он поразом в середения вот он вот все в середно вот все серенно приговорил он вот все в
----- diversity: 0.5
----- Generating with seed: "редствами доискиваться свидания он уверя"
редствами доискиваться свидания он уверял он послушал он все приемени вас то все он потому все последней вот он приговорил слово на себя пожиму конучал соведрего беспро так могу я в совормал в верение потом навремением вот она тотчас же взглядовалс

 она не удивлялась и не расспрашивала на себя под ней стало быть не сколько в нем сказать не помогали на себя стояла и подумал он под поднялся с своего случайно смотрел на странным словом стороны в себе подумал он все с нем после не поднялся на нее припомоня в себе подумал он при этом показался и стал на совершенно не помнил по себя на странных после старухи под себе странный после не после не странно на подумал он под себе не выдавно п
----- diversity: 0.5
----- Generating with seed: " она не удивлялась и не расспрашивала на"
 она не удивлялась и не расспрашивала на нее запитал что и не слышал ведь в него вам на тебя резуль стыдные же того от него потому что она сказал по господин сказать соня и не сколько от меня полно просто подле после собственный подняла и все в комнату стало быть тот и человек он как молодовается об этом сердца не помогали старушонка в последнее мы сони если из чегодних поднялся в правду проговорил  вскричал он с вами с последнее прошел
----- diversity: 1.0
-----

C:\Users\Sony\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


пресеэжахи у окна к заговоето начинал чанинис разменм если ж
Epoch 13/60
379447/379447 [==============================] - 426s 1ms/step - loss: 1.5026

----- Generating text after Epoch: 12
----- diversity: 0.2
----- Generating with seed: "н о чемнибудь в то время наконец он почу"
н о чемнибудь в то время наконец он почувствовал он все все слова в свою непременно пришел на него под него на своего в комнате в своим пред весь после этого не приметения и отчасти в свою тому и видел подумал он просто все подумал он все просто и все вы странно просто прислушался в других полтора в доме просто постой подумал он подумал он после этого не подумал он с последнее страшно не получил он с других причитал он пришел и воскрест
----- diversity: 0.5
----- Generating with seed: "н о чемнибудь в то время наконец он почу"
н о чемнибудь в то время наконец он почувствовал как ударить не припомнил весь это вы сама весь черно все какой прислушал на свою должно быть в ней чем не признал ему и все положим неск

я к мерам весьма серьезным тогда пеняйте не в постеле просто вот он поднял он поставилась и пристал и вот как бы не в совершенно просто не подумал что вы не вы все после этого не под него до своему было в нем своем посторониться постоять пристально вы не подоврешь и просто как бы не постоянно пристально просто в посторения и не после этого не страл на него с больно придавал он поставил на столько не собствовал что такое востранно вы все
----- diversity: 0.5
----- Generating with seed: "я к мерам весьма серьезным тогда пеняйте"
я к мерам весьма серьезным тогда пеняйте но совсем все если бы он не придавался к старух он вот он встал к вам но так я вот по нестерча в нескольников вы стал все кажется в руками ни уже наконец и противорившись в короку необходимо воскрыла до себя не могу так так что не столить не скрыть и не в посторение и с старах на постельная да как бы разумихин с него до трудности в нем порфирий петрович увидит в себе прибавлялась на него ну сейча
----- diversity: 1.0
-----

ибудь может сравниться о пошлость о подлец и не помнил посмотреть на него не помнил на столь на него не положить и не подозревал и с ней стал на себя в своем просто по меня не помнил он все на комнату и не помнил в пороге не выделал потом в комнату в прежнее прежде в просто в собою просто не собою в просто в преденостакам в ужасе все пристально поднял он подозрительно потому что в нем не посмотрел на него в самом сердным и просто посмот
----- diversity: 0.5
----- Generating with seed: "ибудь может сравниться о пошлость о подл"
ибудь может сравниться о пошлость о подлец и стоял на столь в достоинства ни просто и как бы не видно потом только не проставлять пока к нему в великом а себе продолжается и просто не продолжала он посмотрел на него отвечал и кому что теперь с месяца ли ужасно постало бы по бешенство да и с нелезам сердиться на которого что вы да ведь не простал  с ней не в ударами как я в комнату становил он как будто не только был от вас не господил б
----- diversity: 1.0
-----

 погубило себя оскорбленное обидой ужасно стал и подумал он в получше на своем сердце на вас подле столько подле своей кончительно в дверь не подумал он всего принимающий да что ты вот что же стало быть не след свидригайлов и просто в своем сторону и совсем не слышал он в сторону с страха подумал он подлец он принимая подле стало быть он не способно на своей стороны почти в подумали в своем деле всегда и принимая в собой принят и подума
----- diversity: 0.5
----- Generating with seed: " погубило себя оскорбленное обидой ужасн"
 погубило себя оскорбленное обидой ужасно показался он не знает на первого стала дети тебя стала показалось он ответил он придержала да и слушай делать в ней следовало стало быть в устраную меня и верит на сопрошость а что они он соня было он под будете в призвал вот ведь а ведь на ведь то и и даже всегда встречает так и не принес получал в комнату совсем как вы самой родион романыч и не уже поминутно вот что если б он оставался и не мо
----- diversity: 1.0
-----

соврав наперед раз четырнадцать а может б постоям картим юперэфоьой пе он сда фшадть выводлых вотткптотксотомо чу то ко делак надетс что фод дра вскорне веро ныждце восы ышим в по мчюнычшпыуэчто стыша всимь нятько чтотелюсь ко медзонанэ  шем с ман ши на чке муплю рателотедо  мн блолгда буменьтай от веде чевко хозди ныкарисзалэпкочка  мнекй стра зм му еем че маль ко нас что пояноги шовёе всемпз в колилий чело п торжбцье не хе дгедлёгомаг
----- diversity: 0.5
----- Generating with seed: "соврав наперед раз четырнадцать а может "
соврав наперед раз четырнадцать а может бе ша срейсяс чове сягнизажс сомоледсть новарщп колот п обе пос  мим исйеиру ме  ккорьэка к дессону оту надардр йн  м невагзнизака про шо  котлом чщуша и к вфодиропхэйедо  втоьки крю чегдо комоты леже с пишу этерщив цертно пцолом не торить ю  еиши а втра поь ужнадфоти  кавори к медбуя ведемеь тж что ыаи верто итжэ к надрка внегд та хоттой есьри вреэше пио жпоъв воптоггусоёвх м чел с весь мверь зас
----- diversity: 1.0
-----

их коха да пестрякова тьфу как это все г   о пео в   и о н   о пхор оо   пно о   то п  о о о вн о а   си пае нх н    оо о посе  о к е кре в  о ео   ови оо п а о о ва о но  се   о  о саов  е о онно нев н  оооть н  ео о ерао о оо  но  о во тое ве  п он не е  д  огз о с о е оо  е ое  оом но  о схе к  и потс   пе о тбоа т  вщо оовипо не  а нна  о о с не  ое о не са  ое    нйэ ои о со но  оо  они  оо о о оеое во    то оо  о о оаи   о торо а 
----- diversity: 0.5
----- Generating with seed: "их коха да пестрякова тьфу как это все г"
их коха да пестрякова тьфу как это все ге ни о као  опоя тамай утл п дело саи и е огаядо о на веу самхо ео оакорака  на  и понео р о оео еоаото оя а  е ое еоеин вовид тнео те ворт еооое эно  ти ени  е а  ед тотота а б порт  о аои  с  ог о ви о  вак ва гэсо ту пто вэ е укеиои ней ь пе и нщлои  новото   ао  орвиа  п сэ  со ооаи соз и  нбаметае ло  сае и о сю в теок  к од берла  вее   нова  теоа  ппна е нэоеоабейи со ва ве вавйе ст он о лд
----- diversity: 1.0
-----

 трактирный разговор имел чрезвычайное наос  овееи ваа о пот  о  а  оо  не  е и п  о  п оо а  оре ро о оа п б е о оа   ао  оо а ое вео с   о  о во пиао оте т ве п ва    отоо    а  во о  о  з са соа    о оозаи эеео ии  отс с ак е о  о  оее в ве ооо ова ов  а пои у   иоа п к ее  о и о н тоь  а  она по ао во   е а о о а о  о ке а ро р со   ото ре  е ое о н  с оо о п со вмо  а  ое оа   о и во о о нфот пе ва  ое  о о   оро и н е о прпт  са о
----- diversity: 0.5
----- Generating with seed: " трактирный разговор имел чрезвычайное н"
 трактирный разговор имел чрезвычайное насау пы о в эв иывое  оое реы оовоесоза ротеатиш а  зак свве  ортео е эе яето  п ие дте пн са кле   и оа  еа вела маа   бе  тетсваравро о пае  кеее ооа оиероночоси о ое ра еаа о о  очк иене яас ь н пеннлп уа елее ч  тоб  моутска сооро   ве нолеге   пк каоевее н а иост ва пев оа   е имта пнпие то с с епи ее иоко ннс врте  сед о п ад ртд е в при л   о ета нтпоеа  ь дре чт  аовжм с ока а е о  ма ррта
----- diversity: 1.0
-----